TensorFlow-卷积神经网络

In [1]:
#导入NumPy
import numpy as np
#导入TensorFlow
import tensorflow as tf

In [2]:
'''
在TensorFlow中，图像数据集的一种典型表示是[图像数目,长,宽,色彩通道数]的四维张量。
由于这里读入的是灰度图片，色彩通道数为1（彩色 RGB 图像色彩通道数为3），所以我们使用np.expand_dims()函数为图像数据手动在最后添加一维通道。
'''
#图像特征抽取
class MNISTLoader():
    def __init__(self):
        #获取MNIST手写数字数据集
        mnist=tf.keras.datasets.mnist
        #划分数据集
        (self.train_data,self.train_label),(self.test_data,self.test_label)=mnist.load_data()
        #抽取图像特征值
        '''
        MNIST中的图像默认为uint8（0-255的数字），以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道；
        np.expand_dims()用来扩展数组的维度，axis=-1表示数组的最后一个维度。
        '''
        self.train_data=np.expand_dims(self.train_data.astype(np.float32)/255.0,axis=-1)      
        self.test_data=np.expand_dims(self.test_data.astype(np.float32)/255.0,axis=-1)
        #获取目标值
        self.train_label=self.train_label.astype(np.int32) 
        self.test_label=self.test_label.astype(np.int32)
        #样本量
        self.num_train_data,self.num_test_data=self.train_data.shape[0],self.test_data.shape[0]

    #从数据集中随机取出batch_size个元素并返回
    def get_batch(self,batch_size):
        #np.random.randint()在[0,self.num_train_data)范围内抽取batch_size个整数作为索引
        index=np.random.randint(0,self.num_train_data,batch_size)
        #返回索引对应的样本
        return self.train_data[index,:],self.train_label[index]

In [3]:
#卷积神经网络
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        #卷积层1
        self.conv1=tf.keras.layers.Conv2D(
            #卷积核数为32
            filters=32,
            #感受区域形状为[5,5]
            kernel_size=[5,5],
            #滑动步长为1
            strides=1,
            #不够卷积核大小的块就以0填充
            padding='same', 
            #以ReLU函数激活卷积层
            activation=tf.nn.relu  
        )
        #池化层1（池化方法为最大池化，感受区域形状为[2,2]，滑动步长为2） 
        self.pool1=tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2)
        #卷积层2
        self.conv2=tf.keras.layers.Conv2D(
            #卷积核数为32
            filters=64,
            #感受区域形状为[5,5]
            kernel_size=[5,5],
            #不够卷积核大小的块就以0填充
            padding='same',
            #以ReLU函数激活卷积层
            activation=tf.nn.relu
        )
        #池化层2（池化方法为最大池化，感受区域形状为[2,2]，滑动步长为2）
        self.pool2=tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=2)
        #展平层
        self.flatten=tf.keras.layers.Reshape(target_shape=(7*7*64,))
        #全连接层（节点数为1024，以ReLU函数激活）
        self.dense1=tf.keras.layers.Dense(units=1024,activation=tf.nn.relu)
        #输出层（节点数为10，以Softmax函数激活）
        self.dense2=tf.keras.layers.Dense(units=10,activation=tf.nn.softmax)

    def call(self,inputs):
        #卷积层1输出的张量形状为[batch_size,28,28,32]
        x=self.conv1(inputs)                  
        #池化层1输出的张量形状为[batch_size,14,14,32]
        x=self.pool1(x)                       
        #卷积层2输出的张量形状为[batch_size,14,14,64]
        x=self.conv2(x)                       
        #池化层2输出的张量形状为[batch_size,7,7,64]
        x=self.pool2(x)                       
        #展平层输出的张量形状为[batch_size,7*7*64]
        x=self.flatten(x)                     
        #全连接层输出的张量形状为[batch_size,1024]
        x=self.dense1(x)                      
        #输出层输出结果
        output=self.dense2(x)                      
        return output

In [4]:
#设置超参数
#遍历神经网络的次数
num_epochs=5
#每次迭代采用的样本量
batch_size=50
#学习率
learning_rate=0.001

In [5]:
#调用模型
model=CNN()
#调取数据
data_loader=MNISTLoader()
#调用优化器
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [6]:
#迭代次数（//表示整除）
num_batches=int(num_epochs*data_loader.num_train_data//batch_size)
#梯度下降
for batch_index in range(num_batches):
    #训练集的特征值和目标值
    X,y=data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        #预测值
        y_pred=model(X)
        #交叉熵损失函数
        #计算交叉熵
        loss=tf.keras.losses.sparse_categorical_crossentropy(y_true=y,y_pred=y_pred)
        #计算不同维度上交叉熵的平均值
        loss=tf.reduce_mean(loss)
        print("batch %d: loss %f"%(batch_index,loss.numpy()))
    #计算损失函数对变量（参数和偏置）的梯度
    grads=tape.gradient(loss,model.variables)
    #将梯度和变量（参数和偏置）输入优化器
    optimizer.apply_gradients(grads_and_vars=zip(grads,model.variables))

batch 0: loss 2.294902
batch 1: loss 2.155858
batch 2: loss 2.221715
batch 3: loss 2.108884
batch 4: loss 1.892218
batch 5: loss 1.785068
batch 6: loss 1.682334
batch 7: loss 1.379417
batch 8: loss 1.297806
batch 9: loss 1.099741
batch 10: loss 0.910433
batch 11: loss 0.644730
batch 12: loss 1.347262
batch 13: loss 0.554938
batch 14: loss 0.879502
batch 15: loss 0.806543
batch 16: loss 0.619206
batch 17: loss 0.633059
batch 18: loss 0.431193
batch 19: loss 0.499031
batch 20: loss 0.363144
batch 21: loss 0.411001
batch 22: loss 0.470539
batch 23: loss 0.491851
batch 24: loss 0.401950
batch 25: loss 0.475728
batch 26: loss 0.327021
batch 27: loss 0.176375
batch 28: loss 0.444392
batch 29: loss 0.578452
batch 30: loss 0.533668
batch 31: loss 0.470880
batch 32: loss 0.601144
batch 33: loss 0.363236
batch 34: loss 0.684045
batch 35: loss 0.628846
batch 36: loss 0.364421
batch 37: loss 0.259537
batch 38: loss 0.402820
batch 39: loss 0.353576
batch 40: loss 0.253100
batch 41: loss 0.273846
ba

batch 335: loss 0.090510
batch 336: loss 0.049218
batch 337: loss 0.076948
batch 338: loss 0.277749
batch 339: loss 0.024875
batch 340: loss 0.032430
batch 341: loss 0.124162
batch 342: loss 0.093542
batch 343: loss 0.037329
batch 344: loss 0.018216
batch 345: loss 0.013948
batch 346: loss 0.012156
batch 347: loss 0.095532
batch 348: loss 0.076621
batch 349: loss 0.124286
batch 350: loss 0.048510
batch 351: loss 0.339060
batch 352: loss 0.020300
batch 353: loss 0.014348
batch 354: loss 0.028212
batch 355: loss 0.116246
batch 356: loss 0.092487
batch 357: loss 0.231817
batch 358: loss 0.084305
batch 359: loss 0.086613
batch 360: loss 0.100187
batch 361: loss 0.087666
batch 362: loss 0.120195
batch 363: loss 0.087777
batch 364: loss 0.034502
batch 365: loss 0.163867
batch 366: loss 0.114287
batch 367: loss 0.064496
batch 368: loss 0.095107
batch 369: loss 0.053732
batch 370: loss 0.071775
batch 371: loss 0.055631
batch 372: loss 0.018862
batch 373: loss 0.052085
batch 374: loss 0.021889


batch 664: loss 0.091658
batch 665: loss 0.003082
batch 666: loss 0.019683
batch 667: loss 0.008184
batch 668: loss 0.029078
batch 669: loss 0.037504
batch 670: loss 0.053234
batch 671: loss 0.001053
batch 672: loss 0.142515
batch 673: loss 0.006407
batch 674: loss 0.068546
batch 675: loss 0.159209
batch 676: loss 0.080285
batch 677: loss 0.138306
batch 678: loss 0.028731
batch 679: loss 0.081789
batch 680: loss 0.005154
batch 681: loss 0.022575
batch 682: loss 0.141309
batch 683: loss 0.115803
batch 684: loss 0.044062
batch 685: loss 0.093274
batch 686: loss 0.080633
batch 687: loss 0.050722
batch 688: loss 0.022377
batch 689: loss 0.145953
batch 690: loss 0.011735
batch 691: loss 0.016240
batch 692: loss 0.026760
batch 693: loss 0.062699
batch 694: loss 0.025928
batch 695: loss 0.247041
batch 696: loss 0.060052
batch 697: loss 0.052598
batch 698: loss 0.088049
batch 699: loss 0.011171
batch 700: loss 0.037841
batch 701: loss 0.071316
batch 702: loss 0.085150
batch 703: loss 0.087151


batch 993: loss 0.001916
batch 994: loss 0.006403
batch 995: loss 0.013622
batch 996: loss 0.111071
batch 997: loss 0.086909
batch 998: loss 0.011306
batch 999: loss 0.012808
batch 1000: loss 0.004018
batch 1001: loss 0.007687
batch 1002: loss 0.023003
batch 1003: loss 0.010800
batch 1004: loss 0.001137
batch 1005: loss 0.012186
batch 1006: loss 0.055291
batch 1007: loss 0.011455
batch 1008: loss 0.004626
batch 1009: loss 0.056392
batch 1010: loss 0.003064
batch 1011: loss 0.011986
batch 1012: loss 0.051199
batch 1013: loss 0.020681
batch 1014: loss 0.009621
batch 1015: loss 0.003671
batch 1016: loss 0.027401
batch 1017: loss 0.090429
batch 1018: loss 0.003953
batch 1019: loss 0.006133
batch 1020: loss 0.042778
batch 1021: loss 0.003293
batch 1022: loss 0.023755
batch 1023: loss 0.050154
batch 1024: loss 0.041355
batch 1025: loss 0.016168
batch 1026: loss 0.000612
batch 1027: loss 0.029129
batch 1028: loss 0.116499
batch 1029: loss 0.086887
batch 1030: loss 0.016640
batch 1031: loss 0.

batch 1310: loss 0.016742
batch 1311: loss 0.001798
batch 1312: loss 0.007931
batch 1313: loss 0.009634
batch 1314: loss 0.081313
batch 1315: loss 0.131040
batch 1316: loss 0.038312
batch 1317: loss 0.008753
batch 1318: loss 0.041931
batch 1319: loss 0.039350
batch 1320: loss 0.006213
batch 1321: loss 0.019357
batch 1322: loss 0.068858
batch 1323: loss 0.021472
batch 1324: loss 0.003782
batch 1325: loss 0.001720
batch 1326: loss 0.057575
batch 1327: loss 0.001452
batch 1328: loss 0.018451
batch 1329: loss 0.087900
batch 1330: loss 0.042302
batch 1331: loss 0.005967
batch 1332: loss 0.003401
batch 1333: loss 0.148826
batch 1334: loss 0.063519
batch 1335: loss 0.172720
batch 1336: loss 0.072619
batch 1337: loss 0.035953
batch 1338: loss 0.004254
batch 1339: loss 0.016578
batch 1340: loss 0.005213
batch 1341: loss 0.021093
batch 1342: loss 0.002987
batch 1343: loss 0.004595
batch 1344: loss 0.033950
batch 1345: loss 0.037778
batch 1346: loss 0.032725
batch 1347: loss 0.063296
batch 1348: 

...

batch 4487: loss 0.003450
batch 4488: loss 0.059874
batch 4489: loss 0.024484
batch 4490: loss 0.006805
batch 4491: loss 0.000165
batch 4492: loss 0.054007
batch 4493: loss 0.000579
batch 4494: loss 0.021137
batch 4495: loss 0.007227
batch 4496: loss 0.000430
batch 4497: loss 0.000034
batch 4498: loss 0.008116
batch 4499: loss 0.000208
batch 4500: loss 0.000123
batch 4501: loss 0.000029
batch 4502: loss 0.035705
batch 4503: loss 0.065409
batch 4504: loss 0.021269
batch 4505: loss 0.051203
batch 4506: loss 0.000024
batch 4507: loss 0.007281
batch 4508: loss 0.001884
batch 4509: loss 0.001443
batch 4510: loss 0.014529
batch 4511: loss 0.000372
batch 4512: loss 0.000248
batch 4513: loss 0.000055
batch 4514: loss 0.039139
batch 4515: loss 0.006644
batch 4516: loss 0.006645
batch 4517: loss 0.007631
batch 4518: loss 0.001816
batch 4519: loss 0.000295
batch 4520: loss 0.001130
batch 4521: loss 0.001225
batch 4522: loss 0.183032
batch 4523: loss 0.000174
batch 4524: loss 0.000469
batch 4525: 

batch 4805: loss 0.036412
batch 4806: loss 0.000942
batch 4807: loss 0.001859
batch 4808: loss 0.005277
batch 4809: loss 0.000892
batch 4810: loss 0.004482
batch 4811: loss 0.006432
batch 4812: loss 0.001996
batch 4813: loss 0.007894
batch 4814: loss 0.018123
batch 4815: loss 0.000680
batch 4816: loss 0.022719
batch 4817: loss 0.001351
batch 4818: loss 0.010399
batch 4819: loss 0.022011
batch 4820: loss 0.015973
batch 4821: loss 0.000469
batch 4822: loss 0.043292
batch 4823: loss 0.091483
batch 4824: loss 0.033812
batch 4825: loss 0.007737
batch 4826: loss 0.038697
batch 4827: loss 0.155845
batch 4828: loss 0.000120
batch 4829: loss 0.000037
batch 4830: loss 0.002406
batch 4831: loss 0.000329
batch 4832: loss 0.000718
batch 4833: loss 0.003183
batch 4834: loss 0.000840
batch 4835: loss 0.000389
batch 4836: loss 0.079303
batch 4837: loss 0.000717
batch 4838: loss 0.064384
batch 4839: loss 0.013106
batch 4840: loss 0.094554
batch 4841: loss 0.000847
batch 4842: loss 0.001571
batch 4843: 

batch 5123: loss 0.007366
batch 5124: loss 0.014062
batch 5125: loss 0.007254
batch 5126: loss 0.004363
batch 5127: loss 0.005048
batch 5128: loss 0.003614
batch 5129: loss 0.010285
batch 5130: loss 0.000077
batch 5131: loss 0.006239
batch 5132: loss 0.001506
batch 5133: loss 0.001071
batch 5134: loss 0.004915
batch 5135: loss 0.020507
batch 5136: loss 0.000994
batch 5137: loss 0.002211
batch 5138: loss 0.000150
batch 5139: loss 0.037546
batch 5140: loss 0.058220
batch 5141: loss 0.015539
batch 5142: loss 0.014507
batch 5143: loss 0.000781
batch 5144: loss 0.000984
batch 5145: loss 0.010583
batch 5146: loss 0.002807
batch 5147: loss 0.000172
batch 5148: loss 0.004192
batch 5149: loss 0.001451
batch 5150: loss 0.007103
batch 5151: loss 0.000375
batch 5152: loss 0.010033
batch 5153: loss 0.000425
batch 5154: loss 0.000588
batch 5155: loss 0.004711
batch 5156: loss 0.000781
batch 5157: loss 0.086479
batch 5158: loss 0.001366
batch 5159: loss 0.072100
batch 5160: loss 0.000417
batch 5161: 

batch 5441: loss 0.022530
batch 5442: loss 0.004763
batch 5443: loss 0.023282
batch 5444: loss 0.101827
batch 5445: loss 0.000288
batch 5446: loss 0.001411
batch 5447: loss 0.027288
batch 5448: loss 0.002313
batch 5449: loss 0.000022
batch 5450: loss 0.007487
batch 5451: loss 0.003805
batch 5452: loss 0.000203
batch 5453: loss 0.053307
batch 5454: loss 0.005650
batch 5455: loss 0.001926
batch 5456: loss 0.001007
batch 5457: loss 0.022604
batch 5458: loss 0.000170
batch 5459: loss 0.000676
batch 5460: loss 0.116490
batch 5461: loss 0.055142
batch 5462: loss 0.009618
batch 5463: loss 0.004305
batch 5464: loss 0.002680
batch 5465: loss 0.006514
batch 5466: loss 0.001439
batch 5467: loss 0.006511
batch 5468: loss 0.005224
batch 5469: loss 0.002189
batch 5470: loss 0.000664
batch 5471: loss 0.000967
batch 5472: loss 0.003725
batch 5473: loss 0.004678
batch 5474: loss 0.015220
batch 5475: loss 0.000407
batch 5476: loss 0.001871
batch 5477: loss 0.002372
batch 5478: loss 0.000654
batch 5479: 

batch 5759: loss 0.091974
batch 5760: loss 0.000523
batch 5761: loss 0.006398
batch 5762: loss 0.000619
batch 5763: loss 0.000144
batch 5764: loss 0.003631
batch 5765: loss 0.001283
batch 5766: loss 0.001028
batch 5767: loss 0.000204
batch 5768: loss 0.000404
batch 5769: loss 0.000062
batch 5770: loss 0.001504
batch 5771: loss 0.000039
batch 5772: loss 0.000157
batch 5773: loss 0.000036
batch 5774: loss 0.056050
batch 5775: loss 0.004291
batch 5776: loss 0.004060
batch 5777: loss 0.009019
batch 5778: loss 0.032316
batch 5779: loss 0.000354
batch 5780: loss 0.000062
batch 5781: loss 0.001386
batch 5782: loss 0.000258
batch 5783: loss 0.022608
batch 5784: loss 0.001948
batch 5785: loss 0.001530
batch 5786: loss 0.005700
batch 5787: loss 0.044586
batch 5788: loss 0.007948
batch 5789: loss 0.004575
batch 5790: loss 0.021594
batch 5791: loss 0.003361
batch 5792: loss 0.000635
batch 5793: loss 0.000137
batch 5794: loss 0.069211
batch 5795: loss 0.011599
batch 5796: loss 0.032164
batch 5797: 

In [7]:
#调用评估器
sparse_categorical_accuracy=tf.keras.metrics.SparseCategoricalAccuracy()
#迭代次数
num_batches=int(data_loader.num_test_data//batch_size)
for batch_index in range(num_batches):
    #设置索引
    start_index,end_index=batch_index*batch_size,(batch_index+1)*batch_size
    #预测值
    y_pred=model.predict(data_loader.test_data[start_index:end_index])
    #计算精确率
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index],y_pred=y_pred)
print("test accuracy: %f"%sparse_categorical_accuracy.result())

test accuracy: 0.990900
